In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
import re    # for regular expressions
from string import punctuation
from nltk.stem import SnowballStemmer    #if you are brave enough to do stemming
from nltk.corpus import stopwords      #if you want to remove stopwords
from nltk.tokenize import word_tokenize
import string
from collections import Counter
from time import time 
import math

### Data visualization

In [4]:
# Import products

products = pd.read_csv('products_train.csv')
products.head()

,id,locale,title,price,brand,color,size,model,material,author,desc
0,B005ZSSN10,DE,RED DRAGON Amberjack 3 - Steel Tip 22 Gramm Wo...,30.95,RED DRAGON,NaN,NaN,RDD0089,NaN,NaN,Amberjacks Steel Dartpfeile sind verfügbar in ...
1,B08PRYN6LD,DE,Simply Keto Lower Carb* Schokodrops ohne Zucke...,17.90,Simply Keto,NaN,750 g (1er Pack),NaN,NaN,NaN,🌱 NATÜRLICHE SÜSSE DURCH ERYTHRIT - Wir stelle...
2,B09MBZJ48V,DE,"Sennheiser 508377 PC 5.2 Chat, Stilvolles Mult...",68.89,Sennheiser,Multi-Colour,One size,508377,Kunstleder,NaN,3.5 MM BUCHSE - Kann problemlos an Geräte mit ...
3,B08ZN6F26S,DE,AmyBenton Auto ab 1 2 3 ahre - Baby Aufziehbar...,18.99,Amy & Benton,Animal Car,NaN,2008B,aufziehauto 1 jahr,NaN,【Auto aufziehbar】: Drücken Sie einfach leicht ...
4,B094DGRV7D,DE,PLAYMOBIL - 70522 - Cavaliere mit grauem Pony,7.17,PLAYMOBIL,Nicht Zutreffend.,OneSize,70522,Polypropylen,NaN,Inhalt: 1 Stück


In [5]:
# Import sessions

sessions = pd.read_csv('sessions_train.csv')
sessions.head()

,prev_items,next_item,locale
0,['B09W9FND7K' 'B09JSPLN1M'],B09M7GY217,DE
1,['B076THCGSG' 'B007MO8IME' 'B08MF65MLV' 'B001B...,B001B4THSA,DE
2,['B0B1LGXWDS' 'B00AZYORS2' 'B0B1LGXWDS' 'B00AZ...,B0767DTG2Q,DE
3,['B09XMTWDVT' 'B0B4MZZ8MB' 'B0B7HZ2GWX' 'B09XM...,B0B4R9NN4B,DE
4,['B09Y5CSL3T' 'B09Y5DPTXN' 'B09FKD61R8'],B0BGVBKWGZ,DE


In [6]:
# temp_session = sessions.copy()

In [7]:
sessions_eng = sessions[sessions['locale']=='UK']

In [8]:
def clean_items(text):

    text = text[1:-1]
    text = re.findall(r"'([^']*)'", text)

    return text

In [9]:
sessions_eng['prev_items'] = sessions_eng['prev_items'].apply(clean_items)

/var/folders/ys/q3jzjbwd6jv2_whcbxy4d7vh0000gn/T/ipykernel_1696/932254714.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sessions_eng['prev_items'] = sessions_eng['prev_items'].apply(clean_items)


In [10]:
interacted_items = list(sessions_eng['prev_items'])
interacted_items

[['B0BFDL54Y7', 'B0BFDR9X13', 'B07J4WF8VH', 'B07Y21LDJX'],
 ['B07FM2GLNQ', 'B07GZW3P4W'],
 ['B0021L95HU', 'B07DDL77RY', 'B07DDL77RY'],
 ['B0B2WSZYL2', 'B000I8XZ7O'],
 ['B00LW1APOC', 'B00LW1APOC', 'B00OI6NQUI', 'B09HLDN8W1'],
 ['B079DXT1JM', 'B079DZRVDF', 'B079FD3MGB'],
 ['B09X37JZ54', 'B0B9GH9597', 'B0B9GHMSXS', 'B074PQTR4W'],
 ['B094VLF9QV', 'B07ZBKD187'],
 ['B0954WV7VV', 'B09VN6XSDW', 'B0B2R1KL5G'],
 ['B0845XT59N', 'B00474K8SY', 'B008BZVCVM'],
 ['B081KHS89M', 'B08W6W1SP1', 'B00D8CXYXC'],
 ['B0B4V1BH4L', 'B09S642Y37', 'B0BDZXD2NC', 'B0BB61MC84'],
 ['B005C3R4WM', 'B00439VO6I', 'B00J5EVMOA', 'B006MW0G5E'],
 ['1509800697',
  '1509800662',
  'B07XLTQQJN',
  'B07XVP3R4F',
  'B07BR8JQWC',
  'B07BR8JQWC',
  'B002GJFLQ6'],
 ['B0B18M5PWY',
  'B095SZXGG5',
  'B095SX1S12',
  'B0BBSVZ7SR',
  'B072BP5SCL',
  'B06XT1BSD9',
  'B0B993L23X',
  'B08DM17RDG',
  'B08DM1BC3F',
  'B09Q6KCGC4',
  'B09Q73K99D',
  'B0017IMOL2'],
 ['B0B15M68SJ', 'B007531X5Q', 'B096KJDMKF', 'B019VXFEQ6'],
 ['B003ZGE1G0',
  'B09

In [11]:
unique_items = list(set(i for j in interacted_items for i in j))
unique_items

['B08LCPF2MB',
 'B09N9TL5RJ',
 'B09VXQF7CG',
 '000851805X',
 'B09DY9FPJF',
 'B08Y573DJ9',
 'B0881X22R4',
 'B0B63ZCH7T',
 'B01MSQCIVX',
 'B09G2NGKDG',
 'B0BBDDBQJ9',
 'B0922KP5N4',
 'B07V5QDF1L',
 'B08F2SLK68',
 'B09329X214',
 'B09NS5RDVJ',
 'B08FHD2PTL',
 'B09ZQL2W1F',
 'B08VNYMF4L',
 'B0BFBG5F6P',
 'B08H8S1K7V',
 'B00133KRK6',
 'B0BB84BBN6',
 'B09JKSTYWB',
 'B01HESZFKU',
 'B08L41VK5S',
 'B07YG4JJDK',
 'B08YP165F7',
 'B0B8RWL78C',
 'B00ISFJVSG',
 'B01MT6F1E2',
 'B07VVCC4QV',
 'B09KMB9HFH',
 'B07L65R6MK',
 'B07NQCFS9W',
 'B094TVWMZJ',
 'B0B7SKPGRS',
 'B07CT2H8TB',
 'B098B2W5GP',
 'B07DCTJK2Q',
 'B09PY7B57J',
 'B09VT76PRF',
 'B08PVQXFF7',
 'B0B21G7C4V',
 'B09SGQ33GH',
 'B0BDJ8G62P',
 'B0B3JJDK5B',
 'B09X1GRGFH',
 'B07JKJZ454',
 'B08ZNK27HL',
 'B09K7S61P6',
 'B09NFMB1V7',
 'B09JBX7FZD',
 'B00CQ3KRR0',
 'B0B6C64YY9',
 '0307462730',
 'B00C6WR1YS',
 'B07F899GLR',
 'B00D8X0JDE',
 'B088LYQ1MY',
 'B09FPRRXB1',
 'B0811H11T8',
 'B095XT4G9S',
 'B0B2WKT7MN',
 'B0972R3SHP',
 'B09K7Y3HP1',
 'B00VOC75

In [12]:
len(unique_items)

470148

In [13]:
# Check unique locations

products['locale'].unique()

array(['DE', 'JP', 'UK', 'ES', 'FR', 'IT'], dtype=object)

In [14]:
# Exctract the English products only (UK)

products_eng = products[products['locale']=='UK']

In [15]:
products_eng = products_eng[products_eng['id'].isin(unique_items)]

In [16]:
products_eng[products_eng['title'].isna()]

,id,locale,title,price,brand,color,size,model,material,author,desc


In [17]:
# from sklearn.model_selection import train_test_split

In [18]:
# products_eng_used, products_eng_unused = train_test_split(products_eng, train_size=0.5, random_state=42)

In [19]:
# Check the number of products in the English set

# products_eng.size

In [20]:
# Exctract the English sessions only (UK)

# sessions_eng = sessions[sessions['locale']=='UK']

In [21]:
# Check the number of sessions in the English set

# sessions_eng.size

We see that we have more items than distinct sessions

In [22]:
# continue data analysis ...

### Clean

In [23]:
import re
import nltk

In [24]:
titles = np.array(products_eng['title'])
titles = " ".join(titles)

In [25]:
words = re.findall(r'\w+', titles)

In [26]:
words = [w.lower() for w in words]

In [27]:
word_counter = Counter(words)

In [28]:
word_frequencies = np.array(list(word_counter.values()))

In [29]:
word_quartiles = np.quantile(word_frequencies, [0,.45,0.5])
word_quartiles

array([1., 1., 2.])

In [30]:
len(word_frequencies)

148568

In [31]:
word_freq_2_more = [k for k,v in word_counter.items() if float(v) >= 2]

In [32]:
len(word_freq_2_more)

77950

In [33]:
def clean(text, freq_threshold=1):
    
    text = re.findall(r'\b(?:[a-zA-Z]+|\d+)\b', text)

    new_text = []
    for word in text:
        word = word.lower()
        if word_counter[word] > freq_threshold:
            new_text.append(word)
    text = new_text
    
    text = " ".join(text)
    
    return text

In [34]:
products_eng['title'] = products_eng['title'].apply(clean)

In [35]:
products_eng[products_eng['title'].isna()]

,id,locale,title,price,brand,color,size,model,material,author,desc


### Model creation

In [36]:
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split

Problems:
- items with no description

Need to do:
- normalize text (make undercase, no italic, no bold)

In [37]:
import logging 
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

In [38]:
import multiprocessing
cores = multiprocessing.cpu_count()
cores

16

In [39]:
products_eng['title_count'] = products_eng['title'].apply(lambda x: len(x))

In [40]:
products_eng = products_eng[products_eng['title_count'] > 0]

In [41]:
title_lists = np.array(products_eng['title'].apply(word_tokenize).values)

In [42]:
# word2vec parameters
embedding_size = 300
context_window = 5
is_skip_gram = 1
neg_sampling = 20
min_count = 10
alpha = 0.03
doqnsample_freq = 6e-5
min_alpha = 0.0007
epochs = 3
seed = 42

In [43]:
# w2v_model = Word2Vec(
#     vector_size=embedding_size,
#     window=context_window,
#     sg = is_skip_gram,
#     negative = neg_sampling,
#     min_count = min_count,
#     alpha = 0.03,
#     min_alpha = min_alpha,
#     epochs=epochs,
#     sample=doqnsample_freq,
#     workers=cores,
#     seed=42
# )

# train_w2v(w2v_model,titles,epochs)

w2v_model = Word2Vec.load('w2v.model')

INFO - 20:29:02: loading Word2Vec object from w2v.model
INFO - 20:29:02: loading wv recursively from w2v.model.wv.* with mmap=None
INFO - 20:29:02: setting ignored attribute cum_table to None
INFO - 20:29:02: Word2Vec lifecycle event {'fname': 'w2v.model', 'datetime': '2023-05-24T20:29:02.240329', 'gensim': '4.3.1', 'python': '3.10.0 (v3.10.0:b494f5935c, Oct  4 2021, 14:59:20) [Clang 12.0.5 (clang-1205.0.22.11)]', 'platform': 'macOS-13.3.1-x86_64-i386-64bit', 'event': 'loaded'}


In [44]:
def train_w2v(w2v_model, sentences, epochs,save=True,filename=None):
    t = time()
    w2v_model.build_vocab(sentences, progress_per=10_000)

    print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))
    
    t = time()
    w2v_model.train(sentences, total_examples=w2v_model.corpus_count,epochs=epochs,
                    report_delay=1)
    print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))
    
    w2v_model.init_sims(replace=True)
    if save:
        if filename is None:
            w2v_model.save("w2v.model")
        else:
            w2v_model.save(filename)
    return w2v_model
    

In [45]:
print(w2v_model.wv.key_to_index)

{'for': 0, 'with': 1, 'and': 2, '2': 3, 'black': 4, 'pack': 5, 'x': 6, '3': 7, 'of': 8, '1': 9, 'set': 10, '4': 11, '6': 12, '5': 13, 'white': 14, 'kids': 15, 'case': 16, 'to': 17, 'women': 18, 'in': 19, 'party': 20, 'christmas': 21, 'dog': 22, 'light': 23, 'hair': 24, 'halloween': 25, 'cover': 26, 'gift': 27, 'birthday': 28, 'led': 29, 'the': 30, 'home': 31, '8': 32, '12': 33, 'large': 34, 'bag': 35, 'soft': 36, '10': 37, 'blue': 38, 's': 39, 'men': 40, 'girls': 41, 'usb': 42, 'inch': 43, 'storage': 44, 'free': 45, '7': 46, 'baby': 47, 'small': 48, 'iphone': 49, 'kit': 50, 'pro': 51, 'waterproof': 52, 'box': 53, 'card': 54, 'up': 55, 'size': 56, 'gifts': 57, 'toys': 58, 'lights': 59, 'grey': 60, 'bags': 61, '100': 62, 'compatible': 63, 'kitchen': 64, 'outdoor': 65, 'toy': 66, 'cable': 67, 'holder': 68, 'cm': 69, 'non': 70, 'travel': 71, 'anti': 72, 'steel': 73, 'water': 74, 'pcs': 75, 'a': 76, 'wall': 77, 'mini': 78, 'plastic': 79, 'nail': 80, 'adjustable': 81, 'air': 82, 'cat': 83, '

In [46]:
w2v_model.wv['black']

array([-0.02479913, -0.01438221,  0.01639473,  0.06134349, -0.0065829 ,
        0.03433762, -0.01758374, -0.03839943, -0.06464054,  0.02529124,
        0.07795652,  0.05469822,  0.03862044, -0.06817763, -0.02111929,
        0.061957  ,  0.0144649 , -0.01938144, -0.01976247,  0.01537944,
       -0.0683544 , -0.0498211 , -0.03253086, -0.0624342 , -0.01593343,
       -0.08356453, -0.02056943, -0.01393582,  0.09374113, -0.04161102,
        0.02143386, -0.04581588,  0.00699191,  0.0813482 ,  0.02956218,
        0.11270541, -0.00067099, -0.05567363, -0.02372557,  0.02206411,
       -0.0390395 ,  0.04045527,  0.11708733,  0.04557726, -0.05852282,
        0.05098679,  0.04402442,  0.05740783,  0.12733044,  0.11958062,
        0.02551549, -0.04305705, -0.12767158,  0.01434582,  0.09019857,
        0.11316838,  0.0270395 , -0.03625086,  0.03980561, -0.01684682,
        0.00311404, -0.00814364,  0.00151422,  0.06810681, -0.11833242,
        0.06206533,  0.07734399,  0.04552549, -0.03208021,  0.09

In [47]:
def calculate_avg_embeddings(word_lists, model, embedding_size=embedding_size):
    """calculate average embedding for each sentence in a Series/list of sentences"""
    
    avg_embeds = np.zeros((word_lists.shape[0], embedding_size))
    for i,word_ls in enumerate(word_lists):
        sent_avg = np.zeros(embedding_size)
        for w in word_ls:
            if w in model.wv.key_to_index:
                sent_avg += model.wv[w]
                
        if len(word_ls) == 0:
            print(word_ls)
        avg_embeds[i,:] = sent_avg/len(word_ls)
    return avg_embeds

In [48]:
products_eng_embeddings = calculate_avg_embeddings(title_lists, w2v_model)

In [49]:
products_eng_embeddings

array([[ 0.03043979,  0.00527118, -0.01163939, ...,  0.00788403,
         0.01505038, -0.00826885],
       [ 0.00483451, -0.02533822,  0.01465557, ...,  0.0045113 ,
         0.00521037,  0.01057505],
       [ 0.02726747,  0.00673439,  0.01185887, ..., -0.0301495 ,
        -0.02927192, -0.01962404],
       ...,
       [ 0.0620552 , -0.00161691,  0.02615983, ...,  0.00023209,
         0.00954659, -0.01533144],
       [ 0.01603183, -0.03604325,  0.03134473, ...,  0.00891183,
        -0.00704735,  0.00334846],
       [ 0.00014006, -0.01360398, -0.06385279, ...,  0.02401418,
        -0.01613556,  0.00592471]])

In [50]:
titles = products_eng['title']

In [51]:
title_matrix = np.expand_dims(titles,axis=1)

In [52]:
prod_embed_matrix = np.concatenate((title_matrix,products_eng_embeddings),axis=1)

In [53]:
prod_embed_df = pd.DataFrame(prod_embed_matrix)

In [54]:
prod_embed_df['index'] = products_eng['id'].values

In [55]:
cols = prod_embed_df.columns.to_list()
cols = [cols[-1]] + cols[:-1]
prod_embed_df = prod_embed_df[cols]

In [56]:
prod_embed_df.rename({0:"id"},axis=1,inplace=True)

In [73]:
prod_embed_df[prod_embed_df['id']=="0804185328"].values[0][2:].shape

(300,)

In [217]:
# delete zero rows
embed_matrix = np.array(prod_embed_df.iloc[:,2:]).astype(float)
zero_rows = np.all(embed_matrix == 0, axis=1)
zero_row_indices = np.where(zero_rows)[0]

prod_embed_df = prod_embed_df.drop(index=zero_row_indices)

In [75]:
from numpy.linalg import norm

In [221]:
# test case

# big_matrix = np.array(prod_embed_df.iloc[:,2:]).astype(float)
# vec = big_matrix[2,:]
# example_vec = big_matrix[0,:]
# embed_matrix_dot = np.tensordot(big_matrix, vec, axes=1)
# embed_norm_vec = norm(big_matrix, axis=1)*norm(vec)
# similarity_vec = np.divide(embed_matrix_dot,embed_norm_vec)

# a = np.array([1,3,4])
# b= np.array([2,3,4])
# print(np.dot(a,b)/(norm(a)*norm(b)))

# print(similarity_vec[0])

In [236]:
def get_closest_item(item_id, df, num_nearest = 5):
    n_closest_ids = np.array(num_nearest)
    item_vec = df[df['id']==item_id].values[0][2:]
    item_norm = norm(item_vec)
    embed_matrix = np.array(df.iloc[:,2:]).astype(float)
    df.drop(df.columns[2:],axis=1)
    
    embed_matrix_dot = np.tensordot(embed_matrix, item_vec, axes=1)
    embed_norm_vec = norm(embed_matrix, axis=1)*item_norm
    
    similarity_vec = np.divide(embed_matrix_dot,embed_norm_vec)
    
    df['similarity'] = similarity_vec
    df.sort_values(by=['similarity'])
    
    n_closest_ids = df.iloc[:num_nearest,0]

    return n_closest_ids

In [234]:
iid = prod_embed_df.iloc[2,0]
prod_embed_df.iloc[2,1]

'500 easy recipes for every machine both stovetop and electric a cookbook'

### Experiments

In [237]:
n_closest = get_closest_item(iid,prod_embed_df)

In [241]:
prod_embed_df[prod_embed_df['id'].isin(n_closest)]['title'].to_csv('output.csv')